In [2]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [3]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [4]:
EPOCHS =40
BATCH_SIZE = 64

In [5]:
train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST(
        './.data',
        train = True,
        download = True,
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,),(0.3081,))
        ])
    ),
    batch_size = BATCH_SIZE,
    shuffle = True
)
test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST(
        './.data',
        train = False,
        download = True,
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,),(0.3081,))
        ])
    ),
    batch_size = BATCH_SIZE,
    shuffle = True
)

Extracting ./.data\FashionMNIST\raw\train-images-idx3-ubyte.gz to ./.data\FashionMNIST\raw


Extracting ./.data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to ./.data\FashionMNIST\raw


Extracting ./.data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to ./.data\FashionMNIST\raw


Extracting ./.data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to ./.data\FashionMNIST\raw
Processing...

Done!


# tf.nn.max_pool2d( )

    tf.nn.max_pool2d( input, ksize, strides, padding, data_format='NHWC', name=None ) 
      
    - input: Tensor 형식 ( data_format ) .
    - ksize: 입력 Tensor의 각 차원에 대한 윈도우 크기 
    - strides: 슬라이딩 윈도우의 보폭 
    - padding: 'VALID' / 'SAME'
    - data_format: 문자열( 'NHWC', 'NCHW'및 'NCHW_VECT_C' 지원 )
    - name: 조작 이름 ( option ) 
    
    return Tensor의해 지정된 형식 data_format ( The max pooled output tensor )

# view(*args) 
    
    x = F.relu(F.max_pool2d(self.conv1(x),2)) # 컨볼루션 -> 풀링 
    x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)),2)) # 컨볼루션-> 드롭아웃 -> 풀링 
    를 통해서 x는 컨볼루션 계층 2개를 거친 특징 맵이 되었다.
    x는 이미지 처럼 현재 2차원의 모양새를 하고 있음
    특징 맵 이후의 출력을 하는 일반 인공 신경망은 1차원의 입력을 받는다. 
    2차원 특징 맵을 바로 입력으로 넣을 수 없기 때문에 
    view( ) 함수를 통해서 1차원으로 펴준다 ( 차원 축소 )
    
    view()함수에 들어가는 첫번째 입력 -1은 '남는 차원 모두를 뜻하고, 
    320은 x가 가진 원소 개수를 의미한다. 

In [15]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1,10,kernel_size = 5)
        self.conv2 = nn.Conv2d(10,20,kernel_size = 5)
        self.conv2_drop = nn.Dropout2d() #드롭아웃 
        self.fc1 = nn.Linear(320,50) # 신경망
        self.fc2 = nn.Linear(50,10) # 신경망 

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x),2)) # 컨볼루션 -> 풀링  -> 활성화 함수 
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)),2)) # 컨볼루션-> 드롭아웃 -> 풀링  -> 활성화 함수 
        x = x.view(-1,320) # 차원을 2 에서 1로 
        x = F.relu(self.fc1(x)) # 신경망 -> 활성화 함수 
        x = F.dropout(x, training = self.training) #드롭아웃 self.training 값은 train 모드인지 아닌지에 따라 t/f
        x = self.fc2(x) # 신경망 
        return F.log_softmax(x, dim = 1) 
    

In [16]:
model =  Net().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr = 0.01 , momentum = 0.5)

In [26]:
def train (model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 200 == 0 :
            print ('Train Epoch: {} [{}/{} ({:.0f}%)]\t Loss: {:.6f}'
                   .format(epoch, batch_idx*len(data), len(train_loader.dataset),
                        100.*batch_idx/len(train_loader), loss.item()))

In [31]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0 
    correct = 0 
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            test_loss += F.cross_entropy(output, target, reduction = 'sum').item()
            
            pred = output.max(1, keepdim = True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /=len(test_loader.dataset)
    test_accuracy = 100. * correct / len (test_loader.dataset)
    
    return test_loss, test_accuracy

In [ ]:
for epoch in range(1, EPOCHS+1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate( model, test_loader )
    
    print('[{}] Test Loss : {:.4f}, Accuracy: {:.2f}%'
          .format( epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000 (0%)]	 Loss: 0.853285
Train Epoch: 1 [12800/60000 (21%)]	 Loss: 0.776915
Train Epoch: 1 [25600/60000 (43%)]	 Loss: 0.824346
Train Epoch: 1 [38400/60000 (64%)]	 Loss: 0.702100
Train Epoch: 1 [51200/60000 (85%)]	 Loss: 0.646765
[1] Test Loss : 0.5500, Accuracy: 78.48%
Train Epoch: 2 [0/60000 (0%)]	 Loss: 0.801209
Train Epoch: 2 [12800/60000 (21%)]	 Loss: 0.627424
Train Epoch: 2 [25600/60000 (43%)]	 Loss: 0.436741
Train Epoch: 2 [38400/60000 (64%)]	 Loss: 0.728627
Train Epoch: 2 [51200/60000 (85%)]	 Loss: 0.699447
[2] Test Loss : 0.5108, Accuracy: 80.05%
Train Epoch: 3 [0/60000 (0%)]	 Loss: 0.690432
Train Epoch: 3 [12800/60000 (21%)]	 Loss: 0.622497
Train Epoch: 3 [25600/60000 (43%)]	 Loss: 0.540587
Train Epoch: 3 [38400/60000 (64%)]	 Loss: 0.724310
Train Epoch: 3 [51200/60000 (85%)]	 Loss: 0.432773
[3] Test Loss : 0.4936, Accuracy: 80.46%
Train Epoch: 4 [0/60000 (0%)]	 Loss: 0.606510
Train Epoch: 4 [12800/60000 (21%)]	 Loss: 0.675549
Train Epoch: 4 [25600/60000 (4